<a href="https://colab.research.google.com/github/dhrubjun/All-time-best-IPL-team/blob/main/All_time_best_IPL_team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [267]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The team team will consists of 12 player (1 extra player). 
It will have -
1. **5 Batsman**
2. **1 Wicket keeper batsman**
3. **1 all rounder**
4. **4 bowlers (2 spinners and 2 pacers)**

In [263]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = "/content/drive/MyDrive/data/IPL Ball-by-Ball 2008-2020.csv"
df = pd.read_csv(path)


In [ ]:
df.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


Selection of batsman will be on following criteria-
1. **Batting Avg.**
2. **Strike Rate**
3. **Batting average and strike rate at 16th to 20th Over for finisher** 







1.   **Batting Average = (Total runs)/(No. of OUT)** 

2.   **Strike Rate = (Total runs)/(Total no. of balls faced)**



# Calculation of total number of runs for each batsman

















In [ ]:
df_total_run = df.groupby('batsman').sum().sort_values(by='batsman_runs', ascending=False)['batsman_runs']

In [ ]:
df_total_run.head(10)

batsman
V Kohli           5878
SK Raina          5368
DA Warner         5254
RG Sharma         5230
S Dhawan          5197
AB de Villiers    4849
CH Gayle          4772
MS Dhoni          4632
RV Uthappa        4607
G Gambhir         4217
Name: batsman_runs, dtype: int64

# Calculation of total number of balls faced by batsman.

***In calculating total number of balls faced, we will not consider wide balls. ***

In [ ]:
df_total_ball_faced = df[df['extras_type']!='wides'].groupby('batsman').count().sort_values(by='batsman_runs', ascending=False)['batsman_runs']

# Calculation of total number of OUTs

In [ ]:
total_out = df[df['is_wicket']==1].groupby('batsman').count().sort_values(by='is_wicket', ascending=False)['is_wicket']

# **Creating final dataframe for batsman**

In [ ]:
df_batsman = pd.concat([df_total_run,total_out], axis=1)

In [ ]:
df_batsman['Total balls faced'] = df_total_ball_faced

In [ ]:
df_batsman.rename(columns={'batsman_run':'Total Run', 'is_wicket':'Total No.OUT', 'Toatal balls faced':'Total balls faced' }, inplace=True)

**Calculation of batting average**

In [264]:
df_batsman['Batting Average'] = round(df_batsman['batsman_runs']/df_batsman['Total No.OUT'], 2)

**Calculation of batting strike rate**

In [266]:
df_batsman['Batting Strike Rate'] = round((df_batsman['batsman_runs']/df_batsman['Total balls faced'])* 100 , 2)

In [271]:
df_batsman.drop(columns=['Average', 'Strike Rate', 'Points'], inplace=True)

***We will consider only those batsman who has scored more than 1000 runs.***

In [272]:
df_batsman = df_batsman[df_batsman['batsman_runs']>1000]

We will 

In [274]:
df_batsman['Points'] = (df_batsman['Batting Average']*0.7)+(df_batsman['Batting Strike Rate']*0.3)

In [275]:
df_batsman.sort_values(by='Points', ascending=False).head(15)

,batsman_runs,Total No.OUT,Total balls faced,Batting Average,Batting Strike Rate,Points
AD Russell,1517,50.0,832,30.34,182.33,75.937
AB de Villiers,4849,114.0,3192,42.54,151.91,75.351
CH Gayle,4772,116.0,3179,41.14,150.11,73.831
DA Warner,5254,126.0,3712,41.70,141.54,71.652
KL Rahul,2647,62.0,1949,42.69,135.81,70.626
RR Pant,2079,59.0,1368,35.24,151.97,70.259
ML Hayden,1107,27.0,805,41.00,137.52,69.956
JC Buttler,1714,48.0,1146,35.71,149.56,69.865
KS Williamson,1619,41.0,1201,39.49,134.80,68.083
HH Pandya,1349,47.0,847,28.70,159.27,67.871


**Top 20 batsman according to batting average**

In [ ]:
df_batsman.sort_values(by='Average', ascending=False).head(20)['Average']

KL Rahul          42.69
AB de Villiers    42.54
DA Warner         41.70
JP Duminy         41.41
CH Gayle          41.14
ML Hayden         41.00
LMP Simmons       39.96
KS Williamson     39.49
SE Marsh          38.70
MEK Hussey        38.02
MS Dhoni          36.76
V Kohli           36.06
JC Buttler        35.71
RR Pant           35.24
CA Lynn           34.59
KP Pietersen      34.52
F du Plessis      34.36
DA Miller         34.26
S Dhawan          34.19
SK Raina          33.55
Name: Average, dtype: float64

**Top 20 batsman according to batting strike rate**

In [ ]:
df_batsman.sort_values(by='Strike Rate', ascending=False).head(20)['Strike Rate']

AD Russell        182.33
HH Pandya         159.27
V Sehwag          155.44
GJ Maxwell        154.68
RR Pant           151.97
AB de Villiers    151.91
CH Gayle          150.11
KA Pollard        149.88
JC Buttler        149.56
YK Pathan         142.97
DA Warner         141.54
CA Lynn           140.66
DA Miller         138.78
AC Gilchrist      138.39
SR Watson         137.91
ML Hayden         137.52
SK Raina          137.15
Ishan Kishan      136.84
MS Dhoni          136.76
KL Rahul          135.81
Name: Strike Rate, dtype: float64

# Calculation of  batting average and strike rate when the batsman is batting at death overs (16th to 20th Over)

In [276]:
df_death = df[df['over']>15]

In [277]:
df_batsman_run_death = df_death.groupby('batsman').sum().sort_values(by='batsman_runs', ascending=False)['batsman_runs']

In [278]:
df_balls_faced_death  = df_death[df_death['extras_type']!='wides'].groupby('batsman').count().sort_values(by='batsman_runs', ascending=False)['batsman_runs']

In [279]:
df_number_of_out_death = df_death[df_death['is_wicket']==1].groupby('batsman').count().sort_values(by='is_wicket', ascending=False)['is_wicket']

In [280]:
df_death_stat = pd.concat([df_batsman_run_death, df_number_of_out_death], axis=1)

In [281]:
df_death_stat['Total balls faced'] = df_balls_faced_death

In [283]:
df_death_stat['Average'] = round(df_death_stat['batsman_runs']/df_death_stat['is_wicket'], 2)

In [284]:
df_death_stat['Strike Rate'] = round((df_death_stat['batsman_runs']/df_death_stat['Total balls faced'])*100, 2)

In [285]:
df_death_stat = df_death_stat[df_death_stat['batsman_runs']>500]

**Top 10 batsman according to average at 16th to 20th over**

In [286]:
df_death_stat.sort_values(by='Average', ascending=False).head(10)['Average']

AB de Villiers    40.45
JP Duminy         33.90
MS Dhoni          31.55
HH Pandya         28.90
SK Raina          28.77
DA Miller         28.10
V Kohli           26.89
YK Pathan         25.24
AD Russell        24.63
KA Pollard        23.60
Name: Average, dtype: float64

**Top 10 batsman according to average at 16th to 20th over**

In [287]:
df_death_stat.sort_values(by='Strike Rate', ascending=False).head(10)['Strike Rate']

AB de Villiers    233.09
AD Russell        218.03
V Kohli           205.52
RG Sharma         198.27
HH Pandya         192.20
MS Dhoni          189.08
DA Miller         186.12
KA Pollard        185.41
DJ Bravo          185.04
KD Karthik        182.38
Name: Strike Rate, dtype: float64

In [288]:
df_death_stat.head()

,batsman_runs,is_wicket,Total balls faced,Average,Strike Rate
MS Dhoni,2303,73.0,1218,31.55,189.08
KA Pollard,1487,63.0,802,23.60,185.41
AB de Villiers,1254,31.0,538,40.45,233.09
RG Sharma,1144,49.0,577,23.35,198.27
V Kohli,968,36.0,471,26.89,205.52


In [289]:
df_death_stat['Points'] = (df_death_stat['Average']*0.7)+(df_death_stat['Strike Rate']*0.3)

In [330]:
df_death_stat.sort_values(by='Points', ascending=False).head(10)

,batsman_runs,is_wicket,Total balls faced,Average,Strike Rate,Points
AB de Villiers,1254,31.0,538,40.45,233.09,98.242
AD Russell,665,27.0,305,24.63,218.03,82.650
V Kohli,968,36.0,471,26.89,205.52,80.479
MS Dhoni,2303,73.0,1218,31.55,189.08,78.809
HH Pandya,838,29.0,436,28.90,192.20,77.890
JP Duminy,712,21.0,395,33.90,180.25,77.805
RG Sharma,1144,49.0,577,23.35,198.27,75.826
DA Miller,590,21.0,317,28.10,186.12,75.506
KA Pollard,1487,63.0,802,23.60,185.41,72.143
DJ Bravo,742,32.0,401,23.19,185.04,71.745


Selection of bowlers will be on the following criteria-
1. **Bowling average**
2. **Bowling economy**



*   **Bowling average = (No. of runs given)/(No. of wickets taken)**

*   **Bowling economy = (No. of runs given)/(No. of overs bowled)**






In [ ]:
df['dismissal_kind'].unique()

array([nan, 'caught', 'run out', 'bowled', 'lbw', 'retired hurt',
       'stumped', 'caught and bowled', 'hit wicket',
       'obstructing the field'], dtype=object)

# Calculation of Total number of wickets taken

For calculation of total numbers of wickets taken by bowlers following dismissal kinds will not be considered-
1. **Run out**
2. **Retired hurt**
3. **Obstructing the field**

In [ ]:
df1 = df.copy()
df2= df.copy()
df3= df.copy()

In [ ]:
df1 = df1[df1['dismissal_kind'].notnull()]

In [ ]:
df1 = df1[df1['dismissal_kind']!='run out']

In [ ]:
df1 = df1[df1['dismissal_kind']!='retired hurt']

In [ ]:
df1 = df1[df1['dismissal_kind']!='obstructing the field']

In [ ]:
df1['dismissal_kind'].unique()

array(['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled',
       'hit wicket'], dtype=object)

In [ ]:
df1_wickets = df1.groupby('bowler').count().sort_values(by='is_wicket', ascending=False)['is_wicket']

In [ ]:
df1_wickets.head(15)

bowler
SL Malinga         170
A Mishra           160
PP Chawla          156
DJ Bravo           153
Harbhajan Singh    150
R Ashwin           138
B Kumar            136
SP Narine          127
YS Chahal          121
UT Yadav           119
RA Jadeja          114
Sandeep Sharma     109
JJ Bumrah          109
A Nehra            106
R Vinay Kumar      105
Name: is_wicket, dtype: int64

# Calculation of total runs given by bowler

For calculation of total runs given by bowler following extra types will not be considered-
1. **Byes**
2. **Legbyes**

In [ ]:
df2['extras_type'].unique()

array([nan, 'byes', 'wides', 'legbyes', 'noballs', 'penalty'],
      dtype=object)

In [ ]:
df2 = df2[df2['extras_type']!='byes']

In [ ]:
df2 = df2[df2['extras_type']!='legbyes']

In [ ]:
df2_total_runs_given = df2.groupby('bowler').sum().sort_values(by='total_runs', ascending=False)['total_runs']

In [ ]:
df2_total_runs_given.head(15)

bowler
PP Chawla          4263
Harbhajan Singh    3967
A Mishra           3867
DJ Bravo           3798
R Ashwin           3700
UT Yadav           3578
RA Jadeja          3470
SL Malinga         3370
B Kumar            3253
P Kumar            3251
SP Narine          3147
R Vinay Kumar      2966
Z Khan             2782
YS Chahal          2723
SR Watson          2682
Name: total_runs, dtype: int64

# Calculation of total overs bowled by bowler

For calculation of total number of overs bowled by bowler following extra types will not be considered-
1. **Wides**
2. **No balls**

In [ ]:
df3 = df3[df3['extras_type']!='wides']

In [ ]:
df3 = df3[df3['extras_type']!='noballs']

In [ ]:
df3_total_overs = (df3.groupby('bowler').count().sort_values(by='is_wicket', ascending=False)['non_boundary'])/6

In [ ]:
df3_total_overs.head(15)

bowler
Harbhajan Singh    562.333333
PP Chawla          541.666667
R Ashwin           538.333333
A Mishra           526.833333
SL Malinga         471.333333
SP Narine          464.166667
RA Jadeja          452.166667
DJ Bravo           452.000000
B Kumar            449.500000
P Kumar            420.666667
UT Yadav           420.500000
Z Khan             366.666667
DW Steyn           362.666667
YS Chahal          355.000000
R Vinay Kumar      353.500000
Name: non_boundary, dtype: float64

# Creating final dataframe for bowlers

In [ ]:
df_bowler = pd.concat([df1_wickets, df2_total_runs_given, df3_total_overs], axis=1)

In [ ]:
df_bowler.rename(columns={'is_wicket':'No. of Wickets', 'total_runs':'Total runs given', 'non_boundary':'Total overs bowled'}, inplace=True)

In [ ]:
df_bowler.head()

,No. of Wickets,Total runs given,Total overs bowled
SL Malinga,170.0,3370,471.333333
A Mishra,160.0,3867,526.833333
PP Chawla,156.0,4263,541.666667
DJ Bravo,153.0,3798,452.000000
Harbhajan Singh,150.0,3967,562.333333


**Calculation of Bowling Average**

In [ ]:
df_bowler['Bowling Average'] = df_bowler['Total runs given']/df_bowler['No. of Wickets']

**Calculation of bowling economy**

In [ ]:
df_bowler['Bowling Economy'] = df_bowler['Total runs given']/df_bowler['Total overs bowled']

**We will consider only those bowlers who has taken greater than 70 wickets**

In [ ]:
df_bowler1 = df_bowler[df_bowler['No. of Wickets']>70]

**Top 15 bowlers according to bowling average**

In [ ]:
df_bowler1.sort_values(by='Average', ascending=True).head(15)['Average']

SL Malinga        19.823529
Rashid Khan       20.493333
Imran Tahir       21.087500
YS Chahal         22.504132
MM Patel          22.945946
A Nehra           23.537736
JJ Bumrah         23.733945
B Kumar           23.919118
CH Morris         23.987500
A Mishra          24.168750
Sandeep Sharma    24.275229
SP Narine         24.779528
DJ Bravo          24.823529
MJ McClenaghan    25.394366
DW Steyn          25.855670
Name: Average, dtype: float64

**Top 15 bowlers accrding to bowling economy**

In [ ]:
df_bowler1.sort_values(by='Economy', ascending=True).head(15)['Economy']

Rashid Khan        6.247967
SP Narine          6.779892
R Ashwin           6.873065
DW Steyn           6.915441
Harbhajan Singh    7.054535
SL Malinga         7.149929
B Kumar            7.236930
AR Patel           7.298663
A Mishra           7.340082
PP Ojha            7.383886
R Bhatia           7.408313
JJ Bumrah          7.419694
MM Patel           7.518819
Z Khan             7.587273
YS Chahal          7.670423
Name: Economy, dtype: float64

Total points for bowlers will be calculated as follows-


*   **Total points = Bowling Average + Bowling Economy**





In [291]:
df_bowler1['Points'] = df_bowler1.loc[:,'Average'] + df_bowler1.loc[:,'Economy']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [322]:
df_bowler1.sort_values(by='Points', ascending=True).head(10)

,No. of Wickets,Total runs given,Total overs bowled,Average,Economy,Bowling Average,Bowling Economy,Points
Rashid Khan,75.0,1537,246.000000,20.493333,6.247967,20.493333,6.247967,26.741301
SL Malinga,170.0,3370,471.333333,19.823529,7.149929,19.823529,7.149929,26.973459
Imran Tahir,80.0,1687,215.333333,21.087500,7.834365,21.087500,7.834365,28.921865
YS Chahal,121.0,2723,355.000000,22.504132,7.670423,22.504132,7.670423,30.174555
MM Patel,74.0,1698,225.833333,22.945946,7.518819,22.945946,7.518819,30.464765
JJ Bumrah,109.0,2587,348.666667,23.733945,7.419694,23.733945,7.419694,31.153639
B Kumar,136.0,3253,449.500000,23.919118,7.236930,23.919118,7.236930,31.156048
A Nehra,106.0,2495,318.000000,23.537736,7.845912,23.537736,7.845912,31.383648
A Mishra,160.0,3867,526.833333,24.168750,7.340082,24.168750,7.340082,31.508832
SP Narine,127.0,3147,464.166667,24.779528,6.779892,24.779528,6.779892,31.559420


# Calculation of bowling average and bowling economy at death overs (16th to 20th Over)

In [292]:
df4 = df_death.copy()
df5= df_death.copy()
df6= df_death.copy()

**Calculation number of wickets taken in death overs**

In [293]:
df4 = df4[df4['dismissal_kind'].notnull()]

In [294]:
df4 = df4[df4['dismissal_kind']!='run out']

In [295]:
df4 = df4[df4['dismissal_kind']!='retired hurt']

In [296]:
df4 = df4[df4['dismissal_kind']!='obstructing the field']

In [297]:
df_wickets_death = df4.groupby('bowler').count().sort_values(by='is_wicket', ascending=False)['is_wicket']

In [298]:
df_wickets_death.head()

bowler
SL Malinga    90
DJ Bravo      82
B Kumar       66
JJ Bumrah     53
SP Narine     50
Name: is_wicket, dtype: int64

**Calculation of total number of runs given at death**

In [299]:
df5 = df5[df5['extras_type']!='byes']

In [300]:
df5 = df5[df5['extras_type']!='legbyes']

In [301]:
df_total_runs_given_death = df5.groupby('bowler').sum().sort_values(by='total_runs', ascending=False)['total_runs']

In [302]:
df_total_runs_given_death.head()

bowler
DJ Bravo      1567
B Kumar       1232
SL Malinga    1196
JJ Bumrah     1175
UT Yadav      1051
Name: total_runs, dtype: int64

**Calculation of total overs bowled in death**

In [303]:
df6 = df6[df6['extras_type']!='wides']

In [304]:
df6 = df6[df6['extras_type']!='noballs']

In [305]:
df_total_overs_death = (df6.groupby('bowler').count().sort_values(by='is_wicket', ascending=False)['non_boundary'])/6

In [306]:
df_total_overs_death.head()

bowler
DJ Bravo      158.833333
SL Malinga    152.833333
B Kumar       134.333333
JJ Bumrah     131.666667
SP Narine     108.166667
Name: non_boundary, dtype: float64

In [307]:
df_bowler_death = pd.concat([df_wickets_death, df_total_runs_given_death, df_total_overs_death], axis=1)

In [309]:
df_bowler_death.rename(columns={'is_wicket':'No. of Wickets', 'total_runs':'Total runs given', 'non_boundary':'Total overs bowled'}, inplace=True)

In [310]:
df_bowler_death.head()

,No. of Wickets,Total runs given,Total overs bowled
SL Malinga,90.0,1196,152.833333
DJ Bravo,82.0,1567,158.833333
B Kumar,66.0,1232,134.333333
JJ Bumrah,53.0,1175,131.666667
SP Narine,50.0,866,108.166667


In [313]:
df_bowler_death['Bowling Average at death'] = df_bowler_death['Total runs given']/df_bowler_death['No. of Wickets']

In [316]:
df_bowler_death['Bowling Economy at death'] = df_bowler_death['Total runs given']/df_bowler_death['Total overs bowled']

In [320]:
df_bowler_death.drop(columns='Bowling Economy', inplace=True)

In [321]:
df_bowler_death.head()

,No. of Wickets,Total runs given,Total overs bowled,Bowling Average at death,Bowling Economy at death
SL Malinga,90.0,1196,152.833333,13.288889,7.825518
DJ Bravo,82.0,1567,158.833333,19.109756,9.865687
B Kumar,66.0,1232,134.333333,18.666667,9.171216
JJ Bumrah,53.0,1175,131.666667,22.169811,8.924051
SP Narine,50.0,866,108.166667,17.320000,8.006163


In [323]:
df_bowler_death['Points'] = df_bowler_death.loc[:,'Bowling Average at death'] + df_bowler_death.loc[:,'Bowling Economy at death']

**Consider only those bowlers who has taken more than 50 wickets in death overs**

In [328]:
df_bowler_death = df_bowler_death[df_bowler_death['No. of Wickets']>50]

In [329]:
df_bowler_death.sort_values(by='Points', ascending=True).head(10)

,No. of Wickets,Total runs given,Total overs bowled,Bowling Average at death,Bowling Economy at death,Points
SL Malinga,90.0,1196,152.833333,13.288889,7.825518,21.114407
B Kumar,66.0,1232,134.333333,18.666667,9.171216,27.837883
DJ Bravo,82.0,1567,158.833333,19.109756,9.865687,28.975443
JJ Bumrah,53.0,1175,131.666667,22.169811,8.924051,31.093862


# Selection of all rounder

In [ ]:
df_total = pd.concat([df_batsman, df_bowler], axis=1)

In [ ]:
df_total = df_total[df_total['batsman_runs']>1000]

In [ ]:
df_total = df_total[df_total['No. of Wickets']>50]

In [ ]:
df_total.head(10)

,batsman_runs,Total No.OUT,Total balls faced,Average,Strike Rate,Points,No. of Wickets,Total runs given,Total overs bowled,Average,Economy,Bowling Average,Bowling Economy
SR Watson,3874.0,125.0,2809.0,30.99,137.91,63.066,92.0,2682.0,338.166667,29.152174,7.931000,29.152174,7.931000
KA Pollard,3023.0,106.0,2017.0,28.52,149.88,64.928,60.0,1960.0,220.833333,32.666667,8.875472,32.666667,8.875472
JH Kallis,2427.0,89.0,2222.0,27.27,109.23,51.858,65.0,2293.0,290.333333,35.276923,7.897819,35.276923,7.897819
RA Jadeja,2159.0,88.0,1706.0,24.53,126.55,55.136,114.0,3470.0,452.166667,30.438596,7.674161,30.438596,7.674161
AD Russell,1517.0,50.0,832.0,30.34,182.33,75.937,61.0,1713.0,191.166667,28.081967,8.960767,28.081967,8.960767
DJ Bravo,1490.0,65.0,1162.0,22.92,128.23,54.513,153.0,3798.0,452.000000,24.823529,8.402655,24.823529,8.402655
IK Pathan,1139.0,53.0,946.0,21.49,120.40,51.163,80.0,2649.0,340.500000,33.112500,7.779736,33.112500,7.779736


In [ ]:
df_total['Bowling Points'] = df_total['Bowling Average'] + df_total['Bowling Economy']

In [ ]:
df_total.sort_values(by='Bowling Points', ascending=True)['Bowling Points']

DJ Bravo      33.226184
AD Russell    37.042734
SR Watson     37.083174
RA Jadeja     38.112758
IK Pathan     40.892236
KA Pollard    41.542138
JH Kallis     43.174742
Name: Bowling Points, dtype: float64

In [ ]:
df_total.sort_values(by='Points', ascending=False)['Points']

AD Russell    75.937
KA Pollard    64.928
SR Watson     63.066
RA Jadeja     55.136
DJ Bravo      54.513
JH Kallis     51.858
IK Pathan     51.163
Name: Points, dtype: float64

In [ ]:
df_total['Total'] = df_total['Points']-df_total['Bowling Points']

In [ ]:
df_total.sort_values(by='Total', ascending=False)['Total']

AD Russell    38.894266
SR Watson     25.982826
KA Pollard    23.385862
DJ Bravo      21.286816
RA Jadeja     17.023242
IK Pathan     10.270764
JH Kallis      8.683258
Name: Total, dtype: float64